# Домашнее задание № 8

1. Обучить генератор создавать точки, которые будут лежать на графике функции 
 $$y = \frac{sin(x)}{x} - \frac{x}{10}$$    
При выполнении данного задания структура GAN остается той же, но Вам нужно: 
- Сгенерировать настоящие данные    
- Изменить архитектуру дискриминатора и генератора  
- Построить графики

In [ ]:
from torchvision import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torchvision
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform = transforms.Compose([
        transforms.ToTensor()
    ])

dataset = datasets.MNIST(
    '.',
    train=True,
    download=True,
    transform=transform
)

In [ ]:
img_1, label_1 = dataset[10]
img_2, label_2 = dataset[6]

print(f'Расстояние между классами по пикселям {label_1} и {label_2}')
print(torch.mean(torch.square(img_1 - img_2)).item())

grid = torchvision.utils.make_grid([img_1, img_2])
plt.imshow(grid.permute(1, 2, 0));

In [ ]:
img_3, label_3 = dataset[30]

print(f'Расстояние между классами по пикселям {label_1} и {label_3}')
print(torch.mean(torch.square(img_1 - img_3)).item())

grid = torchvision.utils.make_grid([img_1, img_3])
plt.imshow(grid.permute(1, 2, 0));

In [ ]:
class TinyModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=4,
                               kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=4, 
                               out_channels=8,
                               kernel_size=3)
        self.fc1 = nn.Linear(in_features=200,
                             out_features=128)
        self.fc2 = nn.Linear(in_features=128,
                             out_features=num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.relu(x)
        x = x.reshape((-1, 200))
        emb = self.fc1(x)
        x = self.fc2(emb)
        return x, emb

In [ ]:
def train(model, dataloader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.to(device)
    for step, (imgs, labels) in enumerate(dataloader):
        imgs, labels = imgs.to(device), labels.to(device)

        pred, _ = model(imgs)

        loss = criterion(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            print(f'Loss is {loss.item()}')

    return model

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True
)

In [ ]:
model = TinyModel(10)
model = train(model, dataloader)

In [ ]:
model.to('cpu')
_, emb_1 = model(img_1[None])
_, emb_2 = model(img_2[None])
_, emb_3 = model(img_3[None])

In [ ]:
print(f'Расстояние между классами {label_1} и {label_2}')
print(torch.mean(torch.square(emb_1 - emb_2)).item())

print(f'Расстояние между классами {label_1} и {label_3}')
print(torch.mean(torch.square(emb_1 - emb_3)).item())

In [ ]:
anchor = torch.randn(1, 3, requires_grad=True)
positive = torch.randn(1, 3, requires_grad=True)
negative = torch.randn(1, 3, requires_grad=True)

triplet_margin_loss = nn.TripletMarginLoss(margin=1.0)
output = triplet_margin_loss(anchor, positive, negative)

print('anchor: ', anchor)
print('positive: ', positive)
print('negative: ', negative)
print('output: ', output)

In [ ]:
torch.mean(torch.square(anchor - positive))

In [ ]:
torch.mean(torch.square(anchor - negative))